## Blenderbot

This notebook shows how you can chat with Blenderbot through an EMISSOR client. The EMISSOR layer will capture the interaction as a scenario for further analysis. 

### Loading Blenderbot

In [2]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
blender_model = BlenderbotForConditionalGeneration.from_pretrained(mname)
blender_tokenizer = BlenderbotTokenizer.from_pretrained(mname)

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

In [3]:
context_size = 5
def get_answer_from_blender(prompt:str, history_list:[]):
    answer = ""
    sentences = []
    history = ""
    for i, his in enumerate(history):
        if i==context_size:
            break
        history += his +". "
    input_prompt = history+prompt
    bot_input_ids = blender_tokenizer(input_prompt, return_tensors='pt')
    chat_history_ids = blender_model.generate(**bot_input_ids)
    utteranceList = blender_tokenizer.batch_decode(chat_history_ids)
    answer = utteranceList[0].strip('</s>')
    return answer

### Creating an EMISSOR client

In [5]:
from leolani_client import LeolaniChatClient
emissor_path = "./emissor"
HUMAN="Piek"
AGENT="BlenderBot"
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

### Interaction loop

In [10]:
history = []
max_context=50

### First prompt
prompt = "Say whatever you want to say and I will respond. Say \"stop\" or \"bye\" to quit."
print(AGENT + ": " + prompt)
leolaniClient._add_utterance(AGENT, prompt) 
prompt = "Hi "+HUMAN+". How are you doing?"
print(AGENT + ": " + prompt)
leolaniClient._add_utterance(AGENT, prompt) 
history.append(prompt)

utterance = input("\n")
print(HUMAN + ": " + utterance)
leolaniClient._add_utterance(HUMAN, utterance)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    # Create the response from the system and store this as a new signal
    prompt = get_answer_from_blender(utterance, history)
    print('\n\t'+AGENT + ": " + prompt)
    history.append(prompt)
    leolaniClient._add_utterance(AGENT, prompt) 

    utterance = input("\n")
    print(HUMAN + ": " + utterance)
    leolaniClient._add_utterance(HUMAN, utterance)

##### After completion, we save the scenario in the defined emissor folder.
leolaniClient._save_scenario() 

BlenderBot: Say whatever you want to say and I will respond. Say "stop" or "bye" to quit.
BlenderBot: Hi Piek. How are you doing?



 bye


Piek: bye
